In [1]:
import torch
import numpy as np
from msdlib import msd
import time
import pandas as pd
from IPython.core.display import HTML, display
import seaborn as sns
import utils
import tsp_utils
from tsp_utils import Agent
from config import prepare_tsp_config
from rlutils import solve_agent

In [2]:
sns.set()
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)
# warnings.filterwarnings('ignore')

In [3]:
# getting config
config = prepare_tsp_config()

Namespace(T=3, active_search=False, alpha=0.99, batch_size=100, c=10, device='cuda', dimension=2, epoch=10000, greedy=False, hidden1=256, hidden2=128, inference=False, iteration=1, loadpath='trained_model/size_50', lr=0.0001, lr_step=50, lr_step_decay=0.99, maximize=False, n_glimpse=1, num_it=1, num_test=100, print_progress=True, problem_size=50, resume_training=False, save_after=50, savepath='trained_model/size_50', start_index=0)


In [4]:
tsp_instances = tsp_utils.generate_tsp_instances(config, inference=True)
tsp_instances.shape

torch.Size([100, 50, 2])

In [5]:
# solutions from all algorithms
solutions = {}

## Ortools solution

In [6]:
or_solutions, or_time = tsp_utils.solve_tsp_ortools(tsp_instances.numpy())
or_distances = tsp_utils.calculate_distances(tsp_instances.numpy(), or_solutions)
solutions['ortools'] = {'solution': or_solutions, 'distance': or_distances, 'time': or_time}

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:08<00:00, 12.45it/s]


## RL model's solution

In [ ]:
t = time.time()
agent = Agent(config)
for i in range(config.epoch):
    data = tsp_utils.generate_tsp_instances(config)
    agent.learn(data)
agent.plot_learning()
print('elapsed time: ', pd.Timedelta(seconds=time.time()-t))

epoch: 000075| actor_loss: 38.6519,  critic_loss: 4.0624,  avg_distance: 18.231,  avg_critic_out: 18.719            

In [ ]:
evaluate_func = tsp_utils.calculate_distances
solutions['PN_model'] = solve_agent(agent, tsp_instances, config, evaluate_func, 'distance')

In [ ]:
tsp_utils.plot_tsp_solutions(tsp_instances, solutions, 10)

## Results

In [ ]:
comp_table = utils.comparison_table(solutions, keyname='distance', maximize=config.maximize)
comp_table

In [ ]:
utils.result_stats(solutions, 'distance', basename='ortools', other_keys=['time'])

In [ ]:
i = np.random.randint(config.num_test)
print(solutions['ortools']['solution'][i])
print(solutions['PN_model']['solution'][i])